In [202]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from tqdm import trange

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [36]:
disaster_info = pd.read_excel('./data/emdat_public_2023_05_31_query_uid-ldmOwE.xlsx', skiprows=6)

/Users/laurensniesten/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [37]:
# Data cleaning.
disaster_info['Start Date'] = pd.to_datetime(dict(year=disaster_info['Start Year'], month=disaster_info['Start Month'], day=disaster_info['Start Day']))
disaster_info['End Date'] = pd.to_datetime(dict(year=disaster_info['End Year'], month=disaster_info['End Month'], day=disaster_info['End Day']))
disaster_info = disaster_info.drop(columns=['Seq', 'OFDA Response', 'Local Time', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', "Reconstruction Costs ('000 US$)", "Insured Damages ('000 US$)", "Total Damages ('000 US$)", 'CPI', 'Adm Level', 'Admin1 Code', 'Admin2 Code'])

In [38]:
# Omit all natural disasters that took place on dates we do not have news articles on. 
disaster_info = disaster_info[disaster_info['Start Date'] < '2020-04-02'].copy()

In [39]:
# Create a list of the locations (keywords) involved in the natural disaster.
disaster_info['Location List'] = disaster_info['Location'].apply(lambda x: x.replace(' (', '_').replace('), ', '_').replace(', ', '_').replace(')', '').replace('; ', '_').split('_') if type(x) == str else [])

In [40]:
max_total_deaths = disaster_info.loc[disaster_info['Total Deaths'].idxmax()]['Total Deaths']
min_total_deaths = disaster_info.loc[disaster_info['Total Deaths'].idxmin()]['Total Deaths']
max_total_damages_adjusted = disaster_info.loc[disaster_info["Total Damages, Adjusted ('000 US$)"].idxmax()]["Total Damages, Adjusted ('000 US$)"]
min_total_damages_adjusted = disaster_info.loc[disaster_info["Total Damages, Adjusted ('000 US$)"].idxmin()]["Total Damages, Adjusted ('000 US$)"]

print(f'Maximum Total Deaths: {max_total_deaths}')
print(f'Minimum Total Deaths: {min_total_deaths}')
print(f'Maximum Total Damages Adjusted: {max_total_damages_adjusted}')
print(f'Minimum Total Damages Adjusted: {min_total_damages_adjusted}')

Maximum Total Deaths: 4140.0
Minimum Total Deaths: 1.0
Maximum Total Damages Adjusted: 113423011.0
Minimum Total Damages Adjusted: 42.0


In [41]:
news_data = pd.read_csv('./data/all-the-news-2-1.csv') 

In [44]:
news_data['Date'] = pd.to_datetime(dict(year=news_data['year'], month=news_data['month'], day=news_data['day']))
news_data['Time'] = news_data.date.str[-8:]

In [45]:
news_data.drop(columns=['date', 'year', 'month', 'day'], inplace=True)
news_data.rename(columns={'author': 'Author', 'title':'Title', 'url':'Url', 'section':'Section', 'publication':'Publication', 'article': 'Article'}, inplace=True)

In [208]:
news_data_small = news_data.iloc[:100000].copy()

In [209]:
news_data_small['Article'] = news_data_small['Article'].apply(lambda x: x.replace("\'", "").replace("\xa0", " ").replace("  ", " ").replace("\n", " ") if type(x) == str else float("nan"))

In [214]:
disaster_info['mentions_1'] = 0
disaster_info['mentions_2'] = 0
disaster_info['mentions_3'] = 0

## Data Merging

In [215]:
pre_margin, post_margin = 1, 7

In [216]:
for i in range(1000):
    start_date, end_date = disaster_info.iloc[i]['Start Date'], disaster_info.iloc[i]['End Date']
    start_date_adj = start_date - timedelta(days=pre_margin)
    end_date_adj = end_date + timedelta(days=post_margin)
    
    # _________________________________Mentions 1____________________________________________________
    # If event name is mentioned in an article it assumed to talk about the subject.
    if type(disaster_info.iloc[i]['Event Name']) == str:
        kws_1 = conv_dict[disaster_info.iloc[i]['Event Name']]
        print(kws_1)

        candidate_articles = news_data_small[(news_data_small['Date']  < end_date_adj) & (news_data_small['Date']  > start_date_adj)]

        count = 0
        for j, candidate in candidate_articles.iterrows():
            try:
                contains = False
                for kws in kws_1:
                    if all(x in candidate.article.lower() for x in kws):
                        count += 1
                        contains = True
                        continue
                if contains:
                    continue
            except:
                continue
        print(f'count: {count}')
        disaster_info.iloc[i]['mentions_1'] = count
        
    # _________________________________Mentions 2____________________________________________________
    # If any of the locations in the locations list is mentioned together with the type of disaster or 
    # subtype or disaster, the article is counted towards mentions 2.
    
    # _________________________________Mentions 3____________________________________________________
    # If the country is mentioned together with the type of disaster or 
    # subtype or disaster, the article is counted towards mentions 3.

[['cyclone', 'roanu']]
count: 0
[['cyclone', 'winston']]
count: 1
[['fort', 'wildfires'], ['mcmurray', 'wildfires']]


/var/folders/b9/l1t9s3252sl9jwwn1f2_jd3w0000gn/T/ipykernel_4196/638291132.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disaster_info.iloc[i]['mentions_1'] = count


count: 34
[['typhoon', 'butchoy'], ['typhoon', 'nepartak']]
count: 0
[['cyclone', 'Zena']]
count: 0
[['yarloop', 'busfires']]
count: 0
[['storm', 'earl']]
count: 7
[['cholera']]
count: 3
[['storm', 'earl']]
count: 6
[['storm', 'earl']]
count: 6
[['storm', 'earl']]
count: 6
[['hurricane', 'newton']]
count: 0
[['typhoon', 'butchoy'], ['typhoon', 'nepartak']]
count: 0
[['typhoon', 'megi']]
count: 0
[['hurricane', 'matthew']]
count: 22
[['typhoon', 'ferdie'], ['typhoon', 'meranti']]
count: 0
[['storm', 'otto']]
count: 1
[['storm', 'jonas'], ['storm', 'snowzilla']]
count: 8
[['cyclone', 'ulla']]
count: 0
[['cyclone', 'winston']]
count: 1
[['cyclone', 'Zena']]
count: 0
[['typhoon', 'butchoy'], ['typhoon', 'nepartak']]
count: 0
[['sand', 'fire']]
count: 29
[['clayton', 'fire']]
count: 1
[['storm', 'mirinae']]
count: 1
[['dengue']]
count: 7
[['cholera']]
count: 1
[['cholera']]
count: 1
[['yellow', 'fever']]
count: 5
[['typhoon', 'karen'], ['typhoon', 'sarika']]
count: 0
[['typhoon', 'lawin'], 

count: 0
[['hurricane', 'irma']]
count: 38
[['acute', 'watery', 'diarrhea'], ['acute', 'watery', 'diarrhoea']]
count: 0


In [191]:
disaster_info.iloc[:1000]['mentions_1']

0       0
1       0
2       0
3       0
4       0
       ..
1087    0
1088    0
1089    0
1090    0
1092    0
Name: mentions_1, Length: 1000, dtype: int64

In [213]:
conv_dict = conv_dict = {'Cyclone Roanu': [['cyclone', 'roanu']],
             'Cyclone Winston': [['cyclone', 'winston']], 
             'Fort McMurray wildfires': [['fort', 'wildfires'],['mcmurray', 'wildfires']], 
             'Typhoon Butchoy (Nepartak)': [['typhoon','butchoy'],['typhoon', 'nepartak']], 
             'Tropical cyclone Zena': [['cyclone', 'Zena']], 
             'Yarloop busfires': [['yarloop','busfires']], 
             "Tropical storm 'Earl'": [['storm','earl']], 
             'Cholera': [['cholera']], 
             'Hurricane Newton': [['hurricane', 'newton']], 
             'Typhoon Megi': [['typhoon', 'megi']], 
             "Hurricane 'Matthew'": [['hurricane', 'matthew']], 
             'Typhoon Ferdie (Meranti)': [['typhoon', 'ferdie'], ['typhoon', 'meranti']], 
             'Tropical storm Otto': [['storm', 'otto']], 
             'Storm Jonas (Snowzilla)': [['storm','jonas'], ['storm', 'snowzilla']], 
             'Cyclone Ulla': [['cyclone', 'ulla']], 
             'Sand Fire': [['sand', 'fire']], 
             'Clayton Fire': [['clayton', 'fire']], 
             'Storm Mirinae': [['storm', 'mirinae']], 
             'Dengue': [['dengue']], 
             'Yellow fever': [['yellow', 'fever']], 
             'Typhoon Karen (Sarika)': [['typhoon', 'karen'], ['typhoon', 'sarika']], 
             'Typhoon Lawin (Haima)': [['typhoon' ,'lawin'], ['typhoon', 'haima']], 
             'Tropical storm Dianmu': [['storm','dianmu']], 
             'Typhoon Carina (Nida)': [['typhoon','carina'], ['typhoon','nada']], 
             "Tropical depression'16/Hurricane 'Nate'": [['depression', 'nate'], ['hurricane', 'nate']], 
             "Hurricane 'Maria'": [['hurricane','Maria']], 
             'Sir Ivan fire': [['fire','Ivan']], 
             'Gustafsen, Wildwood, Elephant Hill Fires': [['fire','Gustafson'], ['fire','wildwood'], ['fire','elephant', 'hill']], 
             'Debbie': [['Debbie']], 
             "Typhoon 'Hato'": [['typhoon','hate']], 
             "Hurricane 'Irma'": [['hurricane','irma']], 
             'Cyclone Mora': [['cyclone','mora']], 
             'Diphteria': [['diphtheria']], 
             'Tropical Storm Talas': [['storm','talas']], 
             'Typhoon Nesat & Haitang': [['typhoon','nesat'], ['typhoon','haitang']], 
             "Tropical storm 'Pakhar'/'Jolina'": [['storm','pakhar'], ['storm','jolina']], 
             "'Zeus'": [['zeus']], 'Eleanor': [['eleanor']], 
             "Tropical storm 'Alberto'": [['storm','alberto']], 
             'Volcan de Fuego': [['volcan','fuego']], 
             "Typhoon 'Maria' (Gardo)": [['typhoon', 'maria'], ['typhoon', 'gardo']], 
             "Typhoon 'Bebinca'": [['typhoon', 'bebinca']], 
             'Tropical Storm Yagi': [['storm', 'yagi']], 
             'Tropical storm Rumbia': [['storm', 'rumbia']], 
             "Cyclone 'Sagar'": [['cyclone', 'sagar']], 
             "Cyclone 'Josie'": [['cyclone', 'josie']], 
             "'Friederike' (David)": [['friederike'], ['david']], 
             'Influenza': [['influenza']], 
             'Hepatitis E': [['hepatitis e']], 
             "Tropical storm 'Ewiniar'": [['storm', 'ewiniar']], 
             "Tropical storm 'Ampil' (Inday)": [['storm', 'ampil'], ['storm', 'inday']], 
             "Typhoon 'Soulik'": [['typhoon', 'soulik']], 
             'Typhoon Mangkut (Ompong)': [['typhoon', 'mangkut'], ['typhoon', 'ompong']], 
             'Hurricane Michael': [['hurricane', 'michael']], 
             "Hurricane 'Beryl'": [['hurricane', 'beryl']], 
             "Tropical cyclone 'Keni'": [['cyclone', 'keni']], 
             "Storm 'Adrian'": [['storm','adrian']], 
             "Cyclone 'Kenneth'": [['cyclone', 'kenneth']], 
             "Cyclone 'Fani'": [['cyclone', 'fani']], 
             "Cyclone 'Wutip'": [['cyclone', 'wutip']], 
             'Typhoon Chaba (Chyba?)': [['typhoon','chaba'], ['typhoon', 'chyba']], 
             'Tropical storm Mindulle': [['storm', 'mindulle']], 
             'Tropical storm Marce (Tokage)': [['storm','marce'],['storm','tokage']], 
             "Cyclone 'Vardah'": [['cyclone','vardah']], 
             'Typhoon Lionrock': [['typhoon','lionrock']], 
             'Typhoon Malakas': [['typhoon','malakas']], 
             'Typhoon Helen (Megi)': [['typhoon','helen']], 
             'Measles': [['measles']], 'Lassa fever': [['lassa', 'fever']], 
             "Typhoon 'Nina' (Nock-Ten)": [['typhoon','nina'], ['typhoon','nock-ten']], 
             "Typhoon 'Lan'/'Paolo'": [['typhoon','lan'], ['typhoon','paolo']], 
             "Typhoon 'Talim'": [['typhoon','talim']], 
             "Cyclone 'Ockhi'": [['cyclone','ockhi']], 
             'Beatriz': [['beatriz']], 
             "Hurricane 'Franklin'": [['hurricane','franklin']], 
             "Typhoon 'Damrey' / 'Ramil'": [['typhoon','damrey'], ['typhoon','ramil']], 
             "Storm 'Tembin' (Vinta)": [['storm','tembin'],['storm','vinta']], 
             "Tropical storm 'Kai-Tak' (Urduja)": [['storm','kai-tak'], ['storm','urduja']], 
             "Cyclone 'Cempaka'": [['cyclone','cempaka']], 
             'Hurricane Katia': [['hurricane','katia']], 
             "Typhoon 'Doksuri'": [['typhoon','doksuri']], 
             "Cyclone 'Enawo'": [['cyclone','enawo']], 
             "Tropical storm 'Lidia'": [['storm','lidia']], 
             "Cyclone 'Dineo'": [['cyclone','dineo']], 
             'Mt. Agung': [['mt. agung'],['mount agung']], 
             "Typhoon 'Noru'": [['typhoon','noru']], 
             'Marburg': [['marburg']], 
             'Chikungunya': [['chikungunya']], 
             'Plague': [['plague']], 
             'Meningitis': [['meningitis']], 
             'Hepatitis E Virus': [['hepatitis e']], 
             'Tropical depression 02W (Crising)': [['depression','crising']], 
             'Influenza A': [['influenza a']], 
             "Cyclone 'Ava'": [['cyclone','ava']], 
             "Tropical storm 'Basyang' (Sanba)": [['storm','basyang'], ['storm','sanba']], 
             "Tropical depression 'Usman'": [['depression','usman']], 
             "Tropical storm 'Titli'": [['storm','titli']], 
             "Typhoon 'Yutu' (Rosita)": [['typhoon', 'yutu'], ['typhoon', 'rosita']], 
             'Anak Krakatoa': [['anak'], ['krakatoa']], 
             'Mount Etna': [['mount', 'etna'], ['etna']], 
             'Typhoon Jebi': [['typhoon', 'jebi']], 
             'Typhoon Trami': [['typhoon', 'trami']], 
             "Tyhoon 'Son Tinh'": [['typhoon', 'son', 'tinh']], 
             "Tropical storm 'Eliakim'": [['storm', 'eliakim']], 
             "Tropical storm 'Vicente'": [['storm', 'vicente']], 
             "Tropical storm 'Berguitta'": [['storm', 'berguitta']], 
             "Cyclone 'Mekunu'": [['cyclone', 'mekunu']], 
             "Cyclone 'Luban'": [['cyclone', 'luban']], 
             "Cyclone 'Gaja'": [['cyclone', 'gaja']], 
             "Tropical storm 'Phethai'": [['storm', 'pethai']], 
             "Typhoon 'Kong-Rey'": [['typhoon', 'kong-rey']], 
             'Hurricane Willa': [['hurricane', 'willa']], 
             "Tropical cyclone 'Penny'": [['cyclone', 'penny']], 
             'Mt. Mayon': [['mt.', 'mayon'], ['mount', 'mayon']], 
             'Agaton (01W)': [['agaton']], 
             "Storm 'Leslie'": [['storm', 'leslie']], 
             "Storm 'Norma'": [['storm', 'norma']], 
             "Cyclone 'Idai'": [['cyclone', 'idai']], 
             'Acute Encephalitis Syndrome (AES)': [['acute', 'encephalitis', 'syndrome']], 
             "Tropical depression 'Amang' (01W)": [['depression', 'amang']], 
             'Mount Ulawun and Manam': [['mount', 'ulawun', 'manam'], ['mount', 'ulawun'], ['mount', 'manam']], 
             'Hurricane Hermine': [['hurricane', 'hermine']], 
             'Tropical storm Aere': [['storm', 'aere']], 
             'Hurricane Harvey': [['hurricane', 'harvey']], 
             'Acute Watery Diarrhoea': [['acute', 'watery', 'diarrhea'], ['acute', 'watery', 'diarrhoea']], 
             'Wall Fire, Alamo Fire, Whittier Fire': [['wall', 'fire'], ['Alamo', 'fire'], ['Whittier', 'fire']], 
             'Tubbs, Atlas, Nuns Fires': [['tubes', 'atlas', 'nuns', 'fires'], ['tubes', 'nuns', 'fires'], ['nuns', 'fires'], ['tubes', 'fires']], 
             "'Thomas'": [['thomas']], 
             'Mt Monaro': [['mt', 'monaro'], ['mount', 'monaro']], 
             "Tropical cyclone 'Donna'": [['cyclone', 'donna']], 
             'Knysna fire': [['knysna', 'fire']], 'Kilauea': [['kilauea']], 
             'Ambryn': [['ambryn']], 
             'Carr and Mendocino Complex fires': [['carr', 'mendocino', 'fires']], 
             'Cyclone Gita': [['cyclone', 'gita']], 'Hurricane Florence': [['hurricane', 'florence']], 
             'Camp Fire': [['camp', 'fire']], 'Woolsey Fire': [['woolsey', 'fire']], 
             "Tropical depression 'Toraji'": [['depression', 'toraji']], 
             'Ebola': [['ebola']], 
             'Typhoid fever': [['typhoid', 'fever']], 
             'Currowan': [['currowan']], 
             'Tropical cyclone Lekima (Hanna)': [['cyclone', 'lekima'], ['cyclone', 'hanna']], 
             'FL-2019-000131': [['fl-2019-000131']], 
             "Tropical cylone 'Pabuk'": [['cyclone', 'pabuk']], 
             "Tropical cyclone 'Bulbul'": [['cyclone', 'bulbul']], 
             "Tropical cyclone 'Dorian'": [['cyclone', 'dorian']], 
             'Ubinas': [['ubinas']], 
             "Tropical cyclone 'Lingling'": [['cyclone', 'lingling']], 
             "Tropical cyclone 'Mitag'": [['cyclone', 'mitag']], 
             "Cyclone 'Belna'": [['cyclone', 'belna']], 
             "Storm 'Ciara' (Sabine)": [['storm', 'ciara'], ['storm', 'sabine']], 
             "Storm 'Amelie'": [['storm', 'amelie']], 
             "Tropical cylone 'Hagibis'": [['cyclone', 'hagibis']], 
             "Tropical cyclone 'Lorena'": [['cyclone', 'lorena']], 
             "Tropical storm 'Fernand'": [['storm', 'fernand']], 
             'Elsa, Fabien': [['elsa'], ['fabien']], 
             "Tropical cyclone 'Sarai'": [['cyclone', 'sarai']], 
             "Tropical storm 'Tapah'": [['storm', 'tapah']], 
             "Typhoon 'Faxai'": [['typhoon', 'faxai']], 
             "Tropical storm 'Narda'": [['storm', 'narda']], 
             'Whakaari/White Island volcano': [['whakaari', 'volcano'], ['white', 'island', 'volcano']], 
             "Cyclone 'Herold'": [['cyclone', 'herold']], 
             "'Gloria'": [['gloria']], 
             "Cyclone 'Tino'": [['cyclone', 'tino']], 
             "Tropical cyclone 'Kalmaegi' (Ramon)": [['cyclone', 'kalmaegi'], ['cyclone', 'ramon']], 
             "Tropical cyclone 'Phanfone' (Ursula)": [['cyclone', 'phanfone'], ['cyclone', 'ursula']], 
             "Tropical Storm 'Pawan'": [['storm', 'pawan']], 
             "Typhoon 'Matmo'": [['typhoon', 'matmo']], 
             "Tropical storm ' Imelda'": [['storm', 'imelda']], 
             "Tropical cyclone 'Nakri'": [['cyclone', 'nakri']], 
             "Tropical cyclone 'Danas'": [['cyclone', 'danas']], 
             "Tropical cyclone 'Podul'": [['cyclone', 'podul']], 
             "Tropical cyclone 'Kammuri' (Tisoy)": [['cyclone', 'kammuri'], ['cyclone', 'tisoy']], 
             "Tropical cylone 'Barry'": [['cyclone', 'barry']], 
             'Saddleridge fire & Sandalwood fire': [['saddleridge', 'fire'], ['sandalwood', 'fire']], 
             'Kincade Fire': [['kincade', 'fire']], 'Taal volcano': [['taal', 'volcano']]}

In [173]:
conv_dict

{}

In [178]:
a = ['t', 's']
b = 't s'
c = 't r'
if all(x in b for x in a):
    print(True)
if all(x in c for x in a):
    print(False)

True


In [204]:
disaster_info.groupby('Event Name')['Event Name'].count()

Event Name
'Friederike' (David)                    4
'Gloria'                                2
'Thomas'                                1
'Zeus'                                  1
Acute Encephalitis Syndrome (AES)       1
                                       ..
Wall Fire, Alamo Fire, Whittier Fire    1
Whakaari/White Island volcano           1
Woolsey Fire                            1
Yarloop busfires                        1
Yellow fever                            2
Name: Event Name, Length: 178, dtype: int64

In [205]:
disaster_info.groupby(['Event Name'])['Dis No'].count().reset_index(name='count').sort_values(['count'], ascending=False).head(25)

,Event Name,count
13,Cholera,21
52,Hurricane 'Irma',16
40,Dengue,15
53,Hurricane 'Maria',9
82,Storm 'Ciara' (Sabine),9
54,Hurricane 'Matthew',8
116,Tropical depression'16/Hurricane 'Nate',8
69,Measles,6
43,Eleanor,5
0,'Friederike' (David),4


In [207]:
disaster_info[disaster_info['Event Name'] == "Hurricane 'Irma'"]

,Dis No,Year,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,Appeal,Declaration,AID Contribution ('000 US$),Dis Mag Value,Dis Mag Scale,Latitude,Longitude,River Basin,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,"Reconstruction Costs, Adjusted ('000 US$)","Insured Damages, Adjusted ('000 US$)","Total Damages, Adjusted ('000 US$)",Geo Locations,Start Date,End Date,Location List,mentions_1,mentions_2,mentions_3
267,2017-0381-CUB,2017,TC-2017-000132,Natural,Meteorological,Storm,Tropical cyclone,NaN,Hurricane 'Irma',Cuba,CUB,Caribbean,Americas,"Habana del Este, Habana Vieja, Centro Habana, ...",NaN,Flood,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,10.0,NaN,10000000.0,NaN,10000000.0,NaN,238785.0,644720.0,"Camaguey, Ciego De Avila, Cienfuegos, Ciudad D...",2017-09-08,2017-09-10,"[Habana del Este, Habana Vieja, Centro Habana,...",0,0,0
268,2017-0381-DOM,2017,TC-2017-000132,Natural,Meteorological,Storm,Tropical cyclone,NaN,Hurricane 'Irma',Dominican Republic (the),DOM,Caribbean,Americas,"Veron, Higuey (La Altagracia), Samaná (Galera)...",NaN,Flood,NaN,NaN,NaN,NaN,285.0,Kph,NaN,NaN,NaN,NaN,NaN,6300.0,NaN,6300.0,NaN,NaN,NaN,"Azua, Dajabon, Espaillat, La Altagracia, La Ro...",2017-09-06,2017-09-06,"[Veron, Higuey, La Altagracia, Samaná, Galera,...",0,0,0
280,2017-0381-AIA,2017,TC-2017-000132,Natural,Meteorological,Storm,Tropical cyclone,NaN,Hurricane 'Irma',Anguilla,AIA,Caribbean,Americas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,4.0,NaN,15000.0,NaN,15000.0,NaN,7999.0,238785.0,Name Unknown (Adm1).,2017-09-06,2017-09-06,[],0,0,0
286,2017-0381-ATG,2017,TC-2017-000132,Natural,Meteorological,Storm,Tropical cyclone,NaN,Hurricane 'Irma',Antigua and Barbuda,ATG,Caribbean,Americas,"Barbuda, St John and St George districts (Cros...",NaN,NaN,NaN,NaN,NaN,NaN,295.0,Kph,NaN,NaN,NaN,1.0,NaN,1800.0,NaN,1800.0,238785.0,8119.0,298482.0,"Barbuda, St. George, St. John (Adm1).",2017-09-06,2017-09-06,"[Barbuda, St John and St George districts, Cro...",0,0,0
296,2017-0381-BHS,2017,TC-2017-000132,Natural,Meteorological,Storm,Tropical cyclone,NaN,Hurricane 'Irma',Bahamas (the),BHS,Caribbean,Americas,"Inagua, Mayaguana, Crooked Island, Acklins, Lo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,475.0,2388.0,Administrative unit not available (Adm2).,2017-09-08,2017-09-09,"[Inagua, Mayaguana, Crooked Island, Acklins, L...",0,0,0
298,2017-0381-BRB,2017,TC-2017-000132,Natural,Meteorological,Storm,Tropical cyclone,NaN,Hurricane 'Irma',Barbados,BRB,Caribbean,Americas,"St. David's Christ Church, Weston St. James, C...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Christ Church, St. James, St. Joseph (Adm1).",2017-09-08,2017-09-09,"[St. David's Christ Church, Weston St. James, ...",0,0,0
646,2017-0381-HTI,2017,TC-2017-000132,Natural,Meteorological,Storm,Tropical cyclone,NaN,Hurricane 'Irma',Haiti,HTI,Caribbean,Americas,"Caracol (Trou du Nord), Feroer, Malfety (Fort ...",NaN,Flood,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,1.0,17.0,40075.0,NaN,40092.0,NaN,193.0,NaN,"Cap Haitien, Fort Liberte, Ouanaminthe, Trou D...",2017-09-07,2017-09-08,"[Caracol, Trou du Nord, Feroer, Malfety, Fort ...",0,0,0
655,2017-0381-PRI,2017,TC-2017-000132,Natural,Meteorological,Storm,Tropical cyclone,NaN,Hurricane 'Irma',Puerto Rico,PRI,Caribbean,Americas,North. Culebra and Vieques Islands,NaN,Flood,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Aguadilla, Arecibo, Bayamon, Humacao, San Juan...",2017-09-06,2017-09-07,[North. Culebra and Vieques Islands],0,0,0
707,2017-0381-KNA,2017,TC-2017-000132,Natural,Meteorological,Storm,Tropical cyclone,NaN,Hurricane 'Irma',Saint Kitts and Nevis,KNA,Caribbean,Americas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,NaN,NaN,500.0,NaN,500.0,NaN,2746.0,23879.0,"Christ Church Nichola Town, Saint Anne Sandy P...",2017-09-06,2017-09-06,[],0,0,0
1076,